In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'diameter_SAE'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['Diameter']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
# df = pd.concat([dfs.drop('custom_fields', axis=1)], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

7876


In [5]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [6]:
pat_frac=rf'''(?i).{0,10}\d*.?\d+\/\d+.{0,3}diame?t?e?r?.{0,10}|()'''         
df['name'] = df['product_name'].apply(lambda x: re_extract(pat_frac, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pat_frac, str(x)))
# df['custom'] = df['Diameter'].apply(lambda x: re_extract(pat_mm, str(x)))

pat=r'(?i)(.{0,10}\d*.?\d+\/\d+.)|()'
df['matches']=df['Diameter'].apply(lambda x: re_extract(pat, str(x)))
custom=df[df['matches'].astype(str)!='[]']
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|"','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x: re.sub(r'\["','["',str(x)))                                    
matchcustom=custom[['external_id',curation_col]]
matchcustom

external_id Q:diameter_sae
70         64208       ["1/2""]
99         64264       ["3/4""]
110        64277       ["7/8""]
113        64280       ["1/2""]
115        64282       ["5/8""]
120        64287       ["1/2""]
121        64288       ["1/2""]
127        64294       ["1/2""]
130        64297       ["1/2""]
133        64300       ["3/4""]
136        64418       ["1/2""]
138        64421       ["1/2""]
143        64426       ["1/2""]
147        64432       ["1/2""]
155        64187       ["1/2""]
156        64188       ["1/2""]
157        64194       ["5/8""]
159        64170       ["1/2""]
160       641016       ["1/2""]
161        64189       ["1/2""]
162        64181       ["1/2""]
165        64172       ["1/2""]
166        64195     ["11/16""]
167        64196       ["3/4""]
170        64174       ["1/2""]
172       641017       ["5/8""]
173        64197       ["3/4""]
177        64198       ["7/8""]
178        64180       ["1/2""]
179        64191       ["5/8""]
180        64190       ["1/2""]
181        64171       ["1/2""]
182        64173       ["1/2""]
183       641018       ["3/4""]
184        64176       ["1/2""]
185        64175       ["1/2""]
241        62691     ["1-1/2""]
243        62694     ["1-1/4""]
245        62692       ["3/4""]
477        62591        7/16""]
478        62543      ["7/16""]
482        62590         3/4""]
484        62545      ["7/16""]
486        62032       ["3/8""]
488        62525       ["1/4""]
491        62538       ["3/8""]
492        62542       ["3/8""]
493        62530      ["5/16""]
494        62540       ["3/8""]
495        62019       ["1/4""]
496        62537       ["3/8""]
497        62038      ["7/16""]
499        62531      ["5/16""]
501        62536       ["3/8""]
502        62539       ["3/8""]
503        62523       ["1/4""]
504        62532      ["5/16""]
506        62027      ["5/16""]
507        62528      ["5/16""]
508        62034      ["7/16""]
509        62526       ["1/4""]
510        62535       ["3/8""]
511        62037      ["7/16""]
512        62035      ["7/16""]
514        62520      ["3/16""]
516        62024      ["5/16""]
518        62025      ["5/16""]
519        62036      ["7/16""]
520        62028      ["5/16""]
521        62020       ["1/4""]
527        62021       ["1/4""]
529        62014      ["3/16""]
532        62017      ["3/16""]
533        62030       ["3/8""]
534        62029       ["3/8""]
535       621141       ["1/2""]
536        62026      ["5/16""]
539        62018       ["1/4""]
540        62033       ["3/8""]
541       621077      ["3/16""]
542        62031       ["3/8""]
543       621139       ["1/2""]
544        62022      ["5/16""]
545        62010      ["3/16""]
546       621138       ["1/2""]
547       621144       ["1/2""]
549        62011      ["3/16""]
550       621134       ["1/2""]
552        62009      ["3/16""]
553       621142       ["1/2""]
554       621108      ["5/16""]
555       621143       ["1/2""]
556       621135       ["1/2""]
557       621106      ["5/16""]
558       621128       ["3/8""]
559       621132       ["3/8""]
560        62524       ["3/8""]
561        62023      ["5/16""]
562        62521       ["3/8""]
563       621131       ["3/8""]
564       621113      ["5/16""]
565       621127       ["3/8""]
566       621116      ["5/16""]
567       621140       ["1/2""]
568       621111      ["5/16""]
569       621107      ["5/16""]
570       621120       ["3/8""]
571       621115      ["5/16""]
572       621129       ["3/8""]
573       621124       ["3/8""]
574        62522       ["3/8""]
575       621109      ["5/16""]
576       621125       ["3/8""]
577       621098       ["1/4""]
578       621105      ["5/16""]
579       621137       ["1/2""]
580       621112      ["5/16""]
581       621110      ["5/16""]
582       621123       ["3/8""]
583       621136       ["1/2""]
584       621130       ["3/8""]
585       621133       ["1/2""]
587       621099       ["1/4""]
588       621114      ["5/16""]
589  

In [7]:
two=df[df['matches'].astype(str)=='[]']
pat_diam=rf'''(?i)(.?.?.?.?.?.?\d.?.?.?.?.?(?<!head )(?<!pipe )Diam?e?t?e?r?(?! mounting)(?! abrasive ))|((?<!head )(?<!pipe )Diam?e?t?e?r?.?.?\d.?.?.?.?.?.?.?)|()''' 
two['matches'] = two['long_desc'].apply(lambda x: re_extract(pat_diam, str(x)))


# N/A

In [8]:
na=two[two['matches'].astype(str)=='[]']
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
print(len(na)+len(matchcustom))

7841


In [9]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))


# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

_
matchna


In [10]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [11]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [12]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 